In [59]:
import tweepy
import json
import pandas as pd
import re
import nltk
from textblob import TextBlob


api_key = "aJTgi4d1H1zmQNkQuHeualNhP"
api_secrets = "yPFdp2Bbib25r1iPQCqBuPq8B9UzoJbIdpv1jgzEZFEg1eJl6X"
access_token = "1409382627124019204-oPpzVGuCwyFfQTfoocFhgHi68whhog"
access_secret = "esgwpH5gnRfNB0SpiHkO52mZSI5VKKnn8SG2pafEnzj2b"
 
# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key,api_secrets)
auth.set_access_token(access_token,access_secret)
 
api = tweepy.API(auth)

TWEET_OBJECT_PATH = 'tweet-objects/'

 
try:
    api.verify_credentials()
    print('Successful Authentication')
except:
    print('Failed Authentication')

Successful Authentication


In [60]:
def get_ids(filename):
    file = open(filename)
    threads = file.readlines()
    file.close()
    return [thread.replace('\n', '').split(',') for thread in threads]

def get_labels(filename):    
    file = open(filename)
    labels = file.readlines()
    file.close()
    return [label.replace('\n', '') for label in labels]

In [61]:
train_ids = get_ids('train.data.txt')
train_labels = get_labels('train.label.txt')

dev_ids = get_ids('dev.data.txt')
dev_labels = get_labels('dev.label.txt')

test_ids = get_ids('test.data.txt')

print(train_ids[0])
print()
print(train_labels[0])

['1250219300389974016', '1250219116993974272', '1250219437027766273', '1250219620939657216', '1250219777185873922', '1250219894429208577', '1250219998842216448', '1250220115762667520', '1250220272306638848', '1250220389323526146', '1250220527005753344', '1250220791544705025', '1250220987238383616', '1250221140603047937', '1250221275827470336', '1250221402822545410']

nonrumour


In [62]:
flat_train_ids = [id_ for id_list in train_ids for id_ in id_list]
flat_dev_ids = [id_ for id_list in dev_ids for id_ in id_list]
flat_test_ids = [id_ for id_list in test_ids for id_ in id_list]

unique_ids = list(set(flat_train_ids + flat_dev_ids))

In [63]:
# tweet id: full text of tweet if found by api, otherwise text from tweet-objects
tweets_dict = {}

def get_train_tweets_from_api(ids):
    for i in range(0, len(ids), 100):
        split = ids[i:i+100]
        for tweet in api.lookup_statuses(split, tweet_mode='extended'):
            tweets_dict[tweet.id_str] = {}
            tweets_dict[tweet.id_str]['text'] = tweet.full_text
            tweets_dict[tweet.id_str]['user'] = tweet.user.screen_name
            tweets_dict[tweet.id_str]['verified'] = tweet.user.verified
            tweets_dict[tweet.id_str]['likes'] = tweet.favorite_count
            tweets_dict[tweet.id_str]['retweets'] = tweet.retweet_count
            
def get_test_tweets(ids):
    for id_ in ids:
        file = open(f'tweet-objects/{id_}.json')
        tweet_object = json.load(file)
        file.close()
        id_str = tweet_object['id_str']
        tweets_dict[id_str] = {}
        tweets_dict[id_str]['text'] = tweet_object['text']
        tweets_dict[id_str]['user'] = tweet_object['user']['screen_name']
        tweets_dict[id_str]['verified'] = tweet_object['user']['verified']
        tweets_dict[id_str]['likes'] = tweet_object['favorite_count']
        tweets_dict[id_str]['retweets'] = tweet_object['retweet_count']

In [64]:
get_train_tweets_from_api(unique_ids)
get_test_tweets(flat_test_ids)

KeyError: '523589441097433088'

In [70]:
tweet_text = []
tweet_user = []
tweet_verified = []
tweet_likes = []
tweet_retweets = []

for tweet_id in tweets_dict.keys():
    tweet_text.append(tweets_dict[tweet_id]['text'])
    tweet_user.append(tweets_dict[tweet_id]['user'])
    tweet_verified.append(tweets_dict[tweet_id]['verified'])
    tweet_likes.append(tweets_dict[tweet_id]['likes'])
    tweet_retweets.append(tweets_dict[tweet_id]['retweets'])

In [71]:
tweet_ids_df = pd.DataFrame({'id': tweets_dict.keys(), 
                             'text': tweet_text, 
                             'user': tweet_user,
                             'verified': tweet_verified,
                             'likes': tweet_likes,
                             'retweets': tweet_retweets},
                              index=tweets_dict.keys())
tweet_ids_df

,id,text,user,verified,likes,retweets
536827853669941248,536827853669941248,Lego letter from the 1970s still offers a powe...,Independent,True,186,235
527162917138206721,527162917138206721,@nancyhxxx fuck my lyfe,HayleyMadss,False,1,0
537699056806809600,537699056806809600,"@PoloT_TreyG I don't believe it, I want an art...",lenihan16,False,0,0
532297249838354432,532297249838354432,"@nypost HA I tell my sons all the time, ""this ...",RetiredAtFour4,False,0,0
539285886995660803,539285886995660803,@AbbyMartin @PamelaDrew @senorchompers Promine...,ChuckNoyes,False,3,2
...,...,...,...,...,...,...
1222929611232817159,1222929611232817159,@crbabecrab @WHO @DrTedros @WHOWPRO @WHOSEARO ...,96incognito69,False,4,0
1222944075826884608,1222944075826884608,@WHO @DrTedros @WHOWPRO @WHOSEARO @WHO_Europe ...,ellenli_lxw,False,1,0
1239324381215707139,1239324381215707139,Can CoVID-19 be caught from a person who has n...,mugabo_robert,False,1,1
1239324383396716546,1239324383396716546,"However, many people with #COVID-19 experience...",mugabo_robert,False,1,1


In [72]:
api.get_status('1239324385707687938').user.screen_name

'mugabo_robert'

In [97]:
tweet_ids_df.to_csv('tweets_id_text.csv', header=True)

In [98]:
tweet_ids_df = pd.read_csv('tweets_id_text.csv', index_col=0) 
tweet_ids_df.head()

,id,text,user,verified,likes,retweets
536827853669941248,536827853669941248,Lego letter from the 1970s still offers a powe...,Independent,True,186,235
527162917138206721,527162917138206721,@nancyhxxx fuck my lyfe,HayleyMadss,False,1,0
537699056806809600,537699056806809600,"@PoloT_TreyG I don't believe it, I want an art...",lenihan16,False,0,0
532297249838354432,532297249838354432,"@nypost HA I tell my sons all the time, ""this ...",RetiredAtFour4,False,0,0
539285886995660803,539285886995660803,@AbbyMartin @PamelaDrew @senorchompers Promine...,ChuckNoyes,False,3,2


In [99]:
def encode_label(label):
    # rumour: 1, nonrumour: 0
    if label == 'rumour':
        return 1
    return 0


def concatenate_tweets(source, reply_ids):
    concat = source
    
    for id_ in reply_ids:
        try:
            tweet = tweet_ids_df.loc[int(id_), 'text'] # may throw key error
            concat += ' ' + tweet
        except:
            continue
    concat = concat.strip()
    
    return concat


def create_dataframe(ids, labs, include_labels=True):
    source_ids = []
    reply_ids_list = []
    concat_tweets = []
    users = []
    verified = []
    likes = []
    retweets = []
    labels = []
    
    for i in range(len(ids)):
        # add concatenation of source and reply tweets to concat_tweets
        try:
            source = tweet_ids_df.loc[int(ids[i][0]), 'text'] # may throw key error
            if len(ids[i]) > 1:
                concat = concatenate_tweets(source, ids[i][1:])
            concat_tweets.append(concat)
            users.append(tweet_ids_df.loc[int(ids[i][0]), 'user'])
            verified.append(tweet_ids_df.loc[int(ids[i][0]), 'verified'])
            likes.append(tweet_ids_df.loc[int(ids[i][0]), 'likes'])
            retweets.append(tweet_ids_df.loc[int(ids[i][0]), 'retweets'])
        except:
            if include_labels:
                continue # skip instance if source tweet is missing
            else:
                concat_tweets.append('')

        # add id of source tweet to source_ids
        source_ids.append(ids[i][0])

        # add list of ids of reply tweets to reply_ids_list if there are replies
        if len(ids[i]) > 1:
            reply_ids_list.append(ids[i][1:])
        else:
            reply_ids_list.append([])              

        if include_labels:
            # add encoded label to labels (1 for rumour, 0 for nonrumour)
            label = encode_label(labs[i])
            labels.append(label)

    df = pd.DataFrame({'source_id': source_ids, 
                       'reply_ids': reply_ids_list,
                       'concat_tweet': concat_tweets,
                       'user': users, 
                       'verified': verified,
                       'likes': likes,
                       'retweets': retweets})
    
    if include_labels:
        df['label'] = labels
    
    return df

In [100]:
train_full_df = create_dataframe(train_ids, train_labels)
train_full_df.head()

,source_id,reply_ids,concat_tweet,user,verified,likes,retweets,label
0,1250219300389974016,"[1250219116993974272, 1250219437027766273, 125...",5. Can regularly rinsing your nose with saline...,ucoptempe,False,0,0,0
1,554886875303780352,"[554894001946759168, 554959644125167617, 55492...",French police chief killed himself after #Char...,Telegraph,True,43,184,1
2,1237901309011021825,"[1237901311439450112, 1239862502516760577]",Coronavirus disease (COVID-19) advice for the ...,kareem_alnakeeb,False,4,1,0
3,524958128392376320,"[524961934064754688, 524959028061798401, 52495...",Ottawa police confirm that there were multiple...,WSJ,True,23,120,0
4,1239295488677085185,[1239562248990806016],if the primary focus of a government isn't to ...,hoss_bossman,False,6,1,0


In [101]:
dev_full_df = create_dataframe(dev_ids, dev_labels)
dev_full_df.head()

,source_id,reply_ids,concat_tweet,user,verified,likes,retweets,label
0,1249004694950817796,[1249011200068730880],COVID-19 Fact:\nAre hand dryers effective in k...,WeatherBug,False,6,1,0
1,1267552274819227649,"[1270394169836568576, 1270502071175909376]",@atruchecks when can we expect the result of m...,ewart_lynne,False,0,0,0
2,1235238334722699265,"[1235234904281165825, 1235234927937048577, 123...",How does COVID-19 spread? \n\nPeople can catch...,Agali_GCFR,False,0,0,0
3,1248746792914546688,[1248775858120097792],"every news outlet using headlines like,\n\n""ar...",TuckyAalto,False,17,0,0
4,523820806917603328,"[523943560589361152, 524027059346370560, 53048...",Researcher @naskrecki on his encounter with a ...,Harvard,True,71,149,0


In [102]:
test_full_df = create_dataframe(test_ids, None, include_labels=False)
test_full_df

,source_id,reply_ids,concat_tweet,user,verified,likes,retweets
0,1246482832316301319,[1247355493988909056],How Does COVID-19 Spread? https://t.co/TXHDeUp...,WCCO,True,4,0
1,1252279738099433473,"[1251884146403815428, 1252033306100670464, 125...","@brain_warrior I hate to keep saying it, but C...",Kikasitsu,False,0,0
2,1236050255394877440,"[1236050046950481922, 1236050331940855808]",Q. How are COVID-19 and influenza viruses diff...,CovidIreland,False,2,0
3,1235582115900796928,"[1235584239497867275, 1235585067973578752, 123...",Una de les Q&amp;A on coronaviruses de la pàgi...,PerePuigUAB,False,3,4
4,1258787515592572928,"[1258710626676899840, 1258711444075565058, 125...",@_truthpolitics We should absolutely blame the...,klarth,False,0,0
...,...,...,...,...,...,...,...
553,427944719612915712,"[427960532981665792, 427949634032783360, 42794...",Ex-Marlboro man dies from smoking-related dise...,washingtonpost,True,26,139
554,531206167302012929,"[531228826496679936, 531255748157005825, 53120...",Holy shit. Doritos flavored Mountain Dew.\n\nA...,Boogie2988,True,342,181
555,553099685888790528,"[553640547282608128, 557627319322685441, 55310...",Banksy account joins cartoonists support for #...,ThePoke,True,230,350
556,1222928724112396288,"[1222922750546923521, 1222929006967869442, 122...",@DrTedros @WHOWPRO @WHOSEARO @WHO_Europe @paho...,WHO,True,47,19


In [103]:
train_full_df.drop(['source_id','reply_ids'],axis=1,inplace=True)
dev_full_df.drop(['source_id','reply_ids'],axis=1,inplace=True)
test_full_df.drop(['source_id','reply_ids'],axis=1,inplace=True)
train_full_df.head()

,concat_tweet,user,verified,likes,retweets,label
0,5. Can regularly rinsing your nose with saline...,ucoptempe,False,0,0,0
1,French police chief killed himself after #Char...,Telegraph,True,43,184,1
2,Coronavirus disease (COVID-19) advice for the ...,kareem_alnakeeb,False,4,1,0
3,Ottawa police confirm that there were multiple...,WSJ,True,23,120,0
4,if the primary focus of a government isn't to ...,hoss_bossman,False,6,1,0


In [104]:
def preprocess_df(df):
    preprocessed = df
    text = list(preprocessed['concat_tweet'])
    verified = list(preprocessed['verified'])
    sentiment = []
    for i in range(len(text)):
        text[i] = " ".join(filter(lambda x:x[0]!='@', text[i].split()))
        text[i] = " ".join(filter(lambda x:x[0:4]!='http', text[i].split()))
        text[i] = " ".join(filter(lambda x:x[0:4]!='&amp', text[i].split()))
        text[i] = re.sub(r'[^a-zA-Z ]','',text[i])
        text[i] = " ".join(filter(lambda x:x[0:1]!='Q', text[i].split()))
        text[i] = " ".join(filter(lambda x:x[0:1]!='A', text[i].split()))
        sentiment.append(TextBlob(text[i]).sentiment.polarity)
    preprocessed['concat_tweet'] = text
    for j in range(len(verified)):
        if verified[j]:
            verified[j] = 1
        else:
            verified[j] = 0
    preprocessed['verified'] = verified
    preprocessed.insert(1, 'sentiment',sentiment)
    preprocessed.fillna('', inplace=True)
    return preprocessed

In [105]:
train_full_df = preprocess_df(train_full_df)
train_full_df.drop_duplicates(subset=['concat_tweet'][0:100],inplace=True)

dev_full_df = preprocess_df(dev_full_df)
dev_full_df.drop_duplicates(subset=['concat_tweet'][0:100],inplace=True)

test_full_df = preprocess_df(test_full_df)
train_full_df

,concat_tweet,sentiment,user,verified,likes,retweets,label
0,Can regularly rinsing your nose with saline he...,0.173437,ucoptempe,0,0,0,0
1,French police chief killed himself after Charl...,-0.278125,Telegraph,1,43,184,1
2,Coronavirus disease COVID advice for the publi...,0.133333,kareem_alnakeeb,0,4,1,0
3,Ottawa police confirm that there were multiple...,0.133333,WSJ,1,23,120,0
4,if the primary focus of a government isnt to a...,0.400000,hoss_bossman,0,6,1,0
...,...,...,...,...,...,...,...
1560,It cannot be transmitted through goods manufac...,0.040770,sattykrosse,0,0,0,0
1561,Desperate Ted Cruz Claims Planned Parenthood S...,-0.191270,Bipartisanism,1,56,76,1
1562,Thoughts and prayers are not enough Pres Obama...,0.031427,ABC,1,148,108,1
1563,Police have surrounded this building where the...,-0.057692,NBCNews,1,63,176,0


In [106]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, wordnet

tt = TweetTokenizer()
stopwords = set(stopwords.words('english')) #note: stopwords are all in lowercase
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


def tokenize_df(df):
    tokenized_df = df
    tokenized_sentence = []
    for _id, row in tokenized_df.iterrows():
        text = row['concat_tweet']
        # tokenize tweet
        tokens = tt.tokenize(text)
        # convert to lowercase
        tokens = [tok.lower() for tok in tokens]
        # remove stopwords
        tokens = [tok for tok in tokens if tok not in stopwords]
        # lemmatize
        tokens = [lemmatize(tok) for tok in tokens]
        tokenized_sentence.append(tokens)
    tokenized_df.insert(1, 'tokens', tokenized_sentence)
    tokenized_df.drop('concat_tweet',axis=1,inplace=True)
    return tokenized_df

In [107]:
train_full_df = tokenize_df(train_full_df)
train_full_df.head()

,tokens,sentiment,user,verified,likes,retweets,label
0,"[regularly, rinse, nose, saline, help, prevent...",0.173437,ucoptempe,0,0,0,0
1,"[french, police, chief, kill, charliehebdo, at...",-0.278125,Telegraph,1,43,184,1
2,"[coronavirus, disease, covid, advice, public, ...",0.133333,kareem_alnakeeb,0,4,1,0
3,"[ottawa, police, confirm, multiple, suspect, s...",0.133333,WSJ,1,23,120,0
4,"[primary, focus, government, isnt, alleviate, ...",0.400000,hoss_bossman,0,6,1,0


In [108]:
dev_full_df = tokenize_df(dev_full_df)
dev_full_df.head()

,tokens,sentiment,user,verified,likes,retweets,label
0,"[covid, fact, hand, dryer, effective, kill, ne...",0.484091,WeatherBug,0,6,1,0
1,"[expect, result, husband, pendingantibody, tes...",0.000000,ewart_lynne,0,0,0,0
2,"[covid, spread, people, catch, covid, others, ...",0.026257,Agali_GCFR,0,0,0,0
3,"[every, news, outlet, use, headline, like, ant...",-0.056439,TuckyAalto,0,17,0,0
4,"[researcher, encounter, goliath, birdeater, wo...",0.168182,Harvard,1,71,149,0


In [109]:
test_full_df = tokenize_df(test_full_df)
test_full_df

,tokens,sentiment,user,verified,likes,retweets
0,"[covid, spread, thank, wcco, station, trust, m...",0.183333,WCCO,1,4,0
1,"[hate, keep, say, capitalism, implode, without...",-0.160000,Kikasitsu,0,0,0
2,"[covid, influenza, virus, different, covid, co...",0.100000,CovidIreland,0,2,0
3,"[una, de, le, coronaviruses, de, la, pgina, we...",0.068466,PerePuigUAB,0,3,4
4,"[absolutely, blame, politician, whoever, else,...",0.333333,klarth,0,0,0
...,...,...,...,...,...,...
553,"[exmarlboro, man, die, smokingrelated, disease...",0.174937,washingtonpost,1,26,139
554,"[holy, shit, doritos, flavor, mountain, dew, l...",0.043953,Boogie2988,1,342,181
555,"[banksy, account, join, cartoonist, support, c...",0.083333,ThePoke,1,230,350
556,"[member, international, heal, member, adviser,...",0.233333,WHO,1,47,19


In [110]:
def count_tokens(df):
    token_counts = {}
    for token_list in list(df['tokens']):
        for token in token_list:
            if token in token_counts:
                token_counts[token] += 1
            else:
                token_counts[token] = 1
    return {k: v for k, v in sorted(token_counts.items(), key=lambda item: item[1],reverse=True)}
            
unigram_counts = count_tokens(train_full_df)
unigram_counts

{'covid': 2249,
 'coronavirus': 1899,
 'people': 1638,
 'new': 1556,
 'virus': 1071,
 'get': 865,
 'say': 785,
 'like': 783,
 'dont': 645,
 'prevent': 607,
 'make': 589,
 'know': 582,
 'go': 580,
 'kill': 577,
 'spread': 550,
 'one': 535,
 'take': 533,
 'think': 528,
 'need': 519,
 'see': 509,
 'u': 495,
 'disease': 481,
 'would': 474,
 'hand': 473,
 'rt': 447,
 'good': 444,
 'test': 442,
 'person': 430,
 'symptom': 419,
 'protect': 411,
 'time': 407,
 'infect': 398,
 'use': 396,
 'infection': 388,
 'im': 381,
 'help': 372,
 'may': 371,
 'thats': 363,
 'spider': 361,
 'day': 360,
 'die': 351,
 'effective': 348,
 'work': 346,
 'case': 337,
 'cough': 336,
 'give': 335,
 'cause': 326,
 'death': 317,
 'want': 315,
 'someone': 314,
 'treat': 311,
 'report': 310,
 'mask': 306,
 'look': 304,
 'wear': 302,
 'come': 302,
 'shoot': 300,
 'catch': 298,
 'transmit': 295,
 'also': 293,
 'still': 282,
 'way': 278,
 'even': 278,
 'nose': 276,
 'thank': 274,
 'find': 269,
 'right': 269,
 'vaccine': 26

In [144]:
user_counts = {}
repeat_users = []

for user in train_full_df['user']:
    if user == 'mashable':
        if 'u_mashable' in user_counts:
            user_counts['u_mashable'] +=1
        else:
            user_counts['u_mashable'] = 1
    elif user in user_counts:
        user_counts[user] +=1
    else:
        user_counts[user] = 1
        
for k, v in user_counts.items():
    if v > 1:
        repeat_users.append(k)
        
repeat_users

def top_users(df, repeat_users):
    df_new = df
    df_new = df_new.reindex(columns = list(df_new.columns) + repeat_users, fill_value = 0)
    for i, row in df_new.iterrows():
        if row['user'] == 'mashable':
            df_new.at[i, 'u_mashable'] = 1
        elif row['user'] in repeat_users:
            df_new.at[i, row['user']] = 1
    df_new.drop(['user'],axis=1,inplace=True)
    return df_new

In [112]:
def top_n(df, n):
    return list(df.columns)+list(unigram_counts.keys())[0:n]

def df_top_tokens(df, cols):
    df_new = df
    df_new = df_new.reindex(columns=cols, fill_value=0)
    for i, row in df_new.iterrows():
        for token in row['tokens']:
            if token in list(df_new.columns):
                df_new.at[i, token] = 1
    df_new.drop(['tokens'],axis=1,inplace=True)
    return df_new

y_train = train_full_df['label']
y_dev = dev_full_df['label']

train_full_df.drop('label',axis=1,inplace=True)
dev_full_df.drop('label',axis=1,inplace=True)

cols_500 = top_n(train_full_df, 500)
cols_1000 = top_n(train_full_df, 1000)

train_df_500 = df_top_tokens(train_full_df, cols_500)
dev_df_500 = df_top_tokens(dev_full_df, cols_500)
test_df_500 = df_top_tokens(test_full_df, cols_500)

train_df_1000 = df_top_tokens(train_full_df, cols_1000)
dev_df_1000 = df_top_tokens(dev_full_df, cols_1000)
test_df_1000 = df_top_tokens(test_full_df, cols_1000)

train_df_500

,sentiment,user,verified,likes,retweets,covid,coronavirus,people,new,virus,...,employee,reality,lord,attack,law,x,order,continue,post,id
0,0.173437,ucoptempe,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,-0.278125,Telegraph,1,43,184,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0.133333,kareem_alnakeeb,0,4,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.133333,WSJ,1,23,120,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0.400000,hoss_bossman,0,6,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,0.040770,sattykrosse,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1561,-0.191270,Bipartisanism,1,56,76,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1562,0.031427,ABC,1,148,108,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1563,-0.057692,NBCNews,1,63,176,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
#train_df_500_users = top_users(train_df_500, repeat_users)
#train_df_1000_users = top_users(train_df_1000, repeat_users)
train_df_1000_users

,sentiment,verified,likes,retweets,covid,coronavirus,people,new,virus,get,...,Netcare911_sa,ExpediteLlp,OpenParlyZw,washingtonpost,SenatorBiaggi,PennsylvaniaDEP,Jinxie_Al,MANASA96592377,cahwerneck,COVIDNewsByMIB
0,0.586719,0,0.000000,0.000000,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.360938,1,0.005440,0.026096,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.566667,0,0.000506,0.000142,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.566667,1,0.002910,0.017019,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.700000,0,0.000759,0.000142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,0.520385,0,0.000000,0.000000,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1561,0.404365,1,0.007085,0.010779,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1562,0.515714,1,0.018725,0.015317,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1563,0.471154,1,0.007971,0.024961,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
from sklearn.preprocessing import MinMaxScaler

def scale_df(train, dev, test):
    train_scaled = train
    dev_scaled = dev
    test_scaled = test
    for col in ['sentiment','retweets','likes']:
        if col in list(train_scaled.columns):
            sc = MinMaxScaler().fit(train_scaled[col].values.reshape(-1,1))
            train_scaled[col] = sc.transform(train_scaled[col].values.reshape(-1,1))
            dev_scaled[col] = sc.transform(dev_scaled[col].values.reshape(-1,1))
            test_scaled[col] = sc.transform(test_scaled[col].values.reshape(-1,1))
    return train_scaled, dev_scaled, test_scaled

train_df_500, dev_df_500, test_df_500 = scale_df(train_df_500, dev_df_500, test_df_500)
train_df_1000, dev_df_1000, test_df_1000 = scale_df(train_df_1000, dev_df_1000, test_df_1000)
train_df_500

,sentiment,user,verified,likes,retweets,covid,coronavirus,people,new,virus,...,employee,reality,lord,attack,law,x,order,continue,post,id
0,0.586719,ucoptempe,0,0.000000,0.000000,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0.360938,Telegraph,1,0.005440,0.026096,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0.566667,kareem_alnakeeb,0,0.000506,0.000142,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.566667,WSJ,1,0.002910,0.017019,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0.700000,hoss_bossman,0,0.000759,0.000142,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,0.520385,sattykrosse,0,0.000000,0.000000,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1561,0.404365,Bipartisanism,1,0.007085,0.010779,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1562,0.515714,ABC,1,0.018725,0.015317,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1563,0.471154,NBCNews,1,0.007971,0.024961,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from matplotlib import pyplot

model_500 = Sequential()
model_500.add(Dense(502, activation='relu', input_shape=(502,)))
model_500.add(Dropout(0.2))
model_500.add(Dense(128, activation='relu'))
model_500.add(Dropout(0.2))
model_500.add(Dense(64, activation='relu'))
model_500.add(Dropout(0.2))
model_500.add(Dense(1, activation='sigmoid'))
model_500.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model_500.fit(train_df_500, y_train, validation_data=(dev_df_500, y_dev), epochs=4000, verbose=1, callbacks=[es])

_, train_acc = model_500.evaluate(train_df_500, y_train, verbose=0)
_, test_acc = model_500.evaluate(dev_df_500, y_dev, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
model_1000 = Sequential()
model_1000.add(Dense(1002, activation='relu', input_shape=(1002,)))
model_1000.add(Dropout(0.2))
model_1000.add(Dense(512, activation='relu'))
model_1000.add(Dropout(0.2))
model_1000.add(Dense(128, activation='relu'))
model_1000.add(Dropout(0.2))
model_1000.add(Dense(1, activation='sigmoid'))
model_1000.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model_1000.fit(train_df_1000, y_train, validation_data=(dev_df_1000, y_dev), epochs=4000, verbose=1, callbacks=[es])

_, train_acc = model_1000.evaluate(train_df_1000, y_train, verbose=0)
_, test_acc = model_1000.evaluate(dev_df_1000, y_dev, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
def convert_to_binary(x):
    if x < 0.5:
        return 0
    else:
        return 1
    
predictions_500 = [convert_to_binary(y) for y in model_500.predict(test_df_500)]
predictions_1000 = [convert_to_binary(y) for y in model_1000.predict(test_df_1000)]

predictions_1000

In [ ]:
count = 0

for i in range(len(predictions_500)):
    if predictions_500[i] == predictions_1000[i]:
        count+=1
        
count/len(predictions_500)    

In [ ]:
import csv

output_filename = 'nn_predictions_2000.csv'

header = ['Id', 'Predicted']
with open(output_filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i, pred in enumerate(predictions_2000):
        data = [str(i), str(pred)]
        writer.writerow(data)
    f.close()